In [1]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
getwd()

[1] "/mnt/hdd/develop/kaggle/R_sources/house_prices"

In [3]:
source('main.R')
house_prices$helpers$import_libs()

combined_dataset <- house_prices$helpers$load_data()

combined_dataset_fixed <-
    combined_dataset %>%
    (house_prices$missing$categ$replace_with_most_common) %>%
    (house_prices$missing$categ$fix_valid) %>%
    (house_prices$missing$numeric$replace_with_zero)

training_dataset <- combined_dataset_fixed %>% filter(dataSource == 'train')


test_that("should be no NA values in training_dataset", {
    expect_equal(sum(is.na(training_dataset)), 0)
})

In [4]:
categ_data <- 
    training_dataset %>%
    # select categ vars
    select(house_prices$helpers$get_character_colnames(combined_dataset_fixed), SalePrice, -dataSource) %>%
    # drop weak categ vars
    select(-one_of(house_prices$attributes_selection$discared_cat_vars)) %>%
    mutate(
        price_log = log(SalePrice)
    ) %>%
    select(-SalePrice)

categ_data %>% colnames %>% sort

[1] "Alley"         "BldgType"      "BsmtCond"      "BsmtExposure" 
 [5] "BsmtFinType1"  "BsmtQual"      "CentralAir"    "Electrical"   
 [9] "Exterior1st"   "Exterior2nd"   "ExterQual"     "Fence"        
[13] "FireplaceQu"   "Foundation"    "GarageCond"    "GarageFinish" 
[17] "GarageQual"    "GarageType"    "HeatingQC"     "HouseStyle"   
[21] "KitchenQual"   "LandContour"   "LotConfig"     "LotShape"     
[25] "MasVnrType"    "MiscFeature"   "MSSubClass"    "MSZoning"     
[29] "Neighborhood"  "PavedDrive"    "PoolQC"        "price_log"    
[33] "RoofStyle"     "SaleCondition" "SaleType"

In [35]:
source('main.R')
ratings <- house_prices$trans$categ$calc_rating(categ_data, price_log)
ratings %>% head(10)

var,value,rating
Alley,_none_,2.531775
Alley,Grvl,1.500000
Alley,Pave,2.536585
BldgType,1Fam,2.563934
BldgType,2fmCon,1.580645
BldgType,Duplex,1.692308
BldgType,TwnhsE,2.622807
BldgType,Twnhs,1.883721
BsmtCond,TA,2.540809
BsmtCond,Gd,3.046154


In [170]:
config %>%
filter(var == 'Alley') %>%
select(value, rating) -> tmp

mapping <- structure(as.list(tmp$rating), names = as.list(tmp$value))

mapping
mapping[['Grvl']]

$Grvl
[1] 1.5

$`_none_`
[1] 2.531775

$Pave
[1] 2.536585

[1] 1.5

In [199]:
categ_data %>%
select(Alley) %>% 
`[[`(1) -> z

df <- data_frame(map(z, ~mapping[[.]]))

names(df) <- 'Alley'
df %>% head

Alley
2.531775
2.531775
2.531775
2.531775
2.531775
2.531775


In [36]:
categ_data %>% select(-price_log) %>% colnames %>% 
map_dfc(function (col) {
    
    tmp <-
        config %>%
        filter(var == col) %>%
        select(value, rating)
    
    mapping <- structure(as.list(tmp$rating), names = as.list(tmp$value))

    df <- data_frame(
        map(categ_data[, col] [[1]], ~mapping[[.]])
    )
    names(df) <- col
    df
}) %>%
mutate(
    price_log = categ_data$price_log
) %>% 
head

Alley,BldgType,BsmtCond,BsmtExposure,BsmtFinType1,BsmtQual,CentralAir,Electrical,Exterior1st,Exterior2nd,⋯,MiscFeature,MSSubClass,MSZoning,Neighborhood,PavedDrive,PoolQC,RoofStyle,SaleCondition,SaleType,price_log
2.531775,2.563934,2.540809,2.320042,3.258373,3.066343,2.578755,2.59176,3.046602,3.05754,⋯,2.512091,3.448161,2.665508,2.966667,2.584328,2.492085,2.418054,2.451586,2.421468,12.24769
2.531775,2.563934,2.540809,3.283582,2.231818,3.066343,2.578755,2.59176,1.968182,1.96729,⋯,2.512091,2.522388,2.665508,3.454545,2.584328,2.492085,2.418054,2.451586,2.421468,12.10901
2.531775,2.563934,2.540809,2.675439,3.258373,3.066343,2.578755,2.59176,3.046602,3.05754,⋯,2.512091,3.448161,2.665508,2.966667,2.584328,2.492085,2.418054,2.451586,2.421468,12.31717
2.531775,2.563934,3.046154,2.320042,2.231818,1.847458,2.578755,2.59176,1.980583,2.052632,⋯,2.512091,2.366667,2.665508,3.156863,2.584328,2.492085,2.418054,1.920792,2.421468,11.84940
2.531775,2.563934,2.540809,2.900452,3.258373,3.066343,2.578755,2.59176,3.046602,3.05754,⋯,2.512091,3.448161,2.665508,3.97561,2.584328,2.492085,2.418054,2.451586,2.421468,12.42922
2.531775,2.563934,2.540809,2.320042,3.258373,3.066343,2.578755,2.59176,3.046602,3.05754,⋯,2.061224,1.833333,2.665508,2.265306,2.584328,2.492085,2.418054,2.451586,2.421468,11.87060
